# Explore the Caffe Model checkpoint!
Let's get the stuff converted!

In [4]:
import os
import math
import copy
import random
from namedlist import namedlist

import numpy as np
import tensorflow as tf
import cv2

slim = tf.contrib.slim

In [5]:
import sys
sys.path.append('../')

In [6]:
from pprint import pprint

import caffe
import numpy as np
from caffe.proto import caffe_pb2

In [7]:
caffe_filename = '../checkpoints/mobilenets.caffemodel'

caffemodel_params = caffe_pb2.NetParameter()
caffemodel_str = open(caffe_filename, 'rb').read()
caffemodel_params.ParseFromString(caffemodel_str)

In [8]:
layers = caffemodel_params.layer
names = [(i, l.name, l.type, len(l.blobs), l.blobs[0].shape.dim if len(l.blobs) else 0) for i, l in enumerate(layers)]
pprint(names)

[(0, 'data', 'ImageData', 0, 0),
 (1, 'label_data_1_split', 'Split', 0, 0),
 (2, 'conv1', 'Convolution', 1, [32, 3, 3, 3]),
 (3, 'conv1/bn', 'BatchNorm', 3, [32]),
 (4, 'conv1/scale', 'Scale', 2, [32]),
 (5, 'relu1', 'ReLU', 0, 0),
 (6, 'conv2_1/dw', 'Convolution', 1, [32, 1, 3, 3]),
 (7, 'conv2_1/dw/bn', 'BatchNorm', 3, [32]),
 (8, 'conv2_1/dw/scale', 'Scale', 2, [32]),
 (9, 'relu2_1/dw', 'ReLU', 0, 0),
 (10, 'conv2_1/sep', 'Convolution', 1, [64, 32, 1, 1]),
 (11, 'conv2_1/sep/bn', 'BatchNorm', 3, [64]),
 (12, 'conv2_1/sep/scale', 'Scale', 2, [64]),
 (13, 'relu2_1/sep', 'ReLU', 0, 0),
 (14, 'conv2_2/dw', 'Convolution', 1, [64, 1, 3, 3]),
 (15, 'conv2_2/dw/bn', 'BatchNorm', 3, [64]),
 (16, 'conv2_2/dw/scale', 'Scale', 2, [64]),
 (17, 'relu2_2/dw', 'ReLU', 0, 0),
 (18, 'conv2_2/sep', 'Convolution', 1, [128, 64, 1, 1]),
 (19, 'conv2_2/sep/bn', 'BatchNorm', 3, [128]),
 (20, 'conv2_2/sep/scale', 'Scale', 2, [128]),
 (21, 'relu2_2/sep', 'ReLU', 0, 0),
 (22, 'conv3_1/dw', 'Convolution', 1, [

In [16]:
idx = 7
layer = layers[idx]
print(layer.type)

j = 2
a = np.array(layer.blobs[j].data)
s = layer.blobs[j].shape.dim

print(s, a)

BatchNorm
[1] [ 1.]


In [10]:
layer.blobs[1].data

[-3.6450198876991635e-06, 0.692308247089386, 0.41665300726890564, 0.421708881855011, 0.2662075459957123, 0.3960053622722626, 0.7624814510345459, 0.4683070480823517, 1.1972514357694308e-07, -1.2171186725140615e-08, 1.2954884276439316e-08, 0.6677316427230835, 0.4605679512023926, 1.5962989152740192e-08, 1.1254932097415349e-08, 0.3987671136856079, 0.47877001762390137, 0.38756898045539856, 0.5033817291259766, 0.36759766936302185, -0.29059037566185, 0.2722242474555969, -1.4396533742910833e-06, 0.4062250554561615, 0.4826686978340149, -1.3022541907048435e-06, -6.403853944902949e-07, 2.4551898292202168e-08, 0.3841036260128021, 6.597526969187584e-10, 0.19825950264930725, 0.5088995099067688]

# Try to get it converted...


In [8]:
import tensorflow as tf

isess = tf.InteractiveSession()

In [9]:
from nets import mobilenets

In [10]:
# Load caffe model.
caffe_filename = '../checkpoints/mobilenets.caffemodel'
caffemodel_params = caffe_pb2.NetParameter() 
caffemodel_params.ParseFromString(open(caffe_filename, 'rb').read())

In [11]:
# TF stuff!
tf.logging.set_verbosity(tf.logging.INFO)
global_step = slim.create_global_step()

In [12]:
# MobileNets TF model.
imgsize = mobilenets.mobilenets.default_image_size
shape = (1, imgsize, imgsize, 3)
img_input = tf.placeholder(shape=shape, dtype=tf.float32)
# Create MobileNets net.
reuse = True if 'outnet' in locals() else None
with slim.arg_scope(mobilenets.mobilenets_arg_scope(data_format='NHWC')):
    outnet = mobilenets.mobilenets(img_input, num_classes=1000, is_training=False, reuse=reuse)

# Initialize variables.
isess.run(tf.global_variables_initializer())
isess.run(tf.local_variables_initializer())


In [13]:
def get_conv_weights(idx, caffemodel_params, bgr_to_rgb=True, dtype=tf.float32):
    # Find the correct layer.
    layers = caffemodel_params.layer
    i = -1
    j = 0    
    while j < len(layers):
        if layers[j].type == 'Convolution':
            i += 1
        if i == idx:
            break
        j += 1
    layer = layers[j]
    # Get weights...
    w = np.array(layer.blobs[0].data)
    w = np.reshape(w, layer.blobs[0].shape.dim)
    # w = np.transpose(w, (1, 0, 2, 3))
    w = np.transpose(w, (2, 3, 1, 0))
    if bgr_to_rgb == 1 and w.shape[2] == 3:
        print('Convert BGR to RGB in convolution layer:', layer.name)
        w[:, :, (0, 1, 2)] = w[:, :, (2, 1, 0)]
        bgr_to_rgb += 1
    print('Load weights from convolution layer:', layer.name, w.shape)
    return w
#     return tf.cast(w, dtype)
    

idx = 0
var_names_to_values = {}
for v in tf.global_variables():
    name = v.op.name
    if 'conv' in name and 'weights' in name:
        print('Assign variable:', v.op.name, v.get_shape())
        var_names_to_values[v.op.name] = get_conv_weights(idx, caffemodel_params)
        idx += 1

# Assign weights...
assign_op, feed_dict = tf.contrib.framework.assign_from_values(var_names_to_values)
isess.run(assign_op, feed_dict=feed_dict)

Assign variable: MobileNets/conv1/weights (3, 3, 3, 32)
Convert BGR to RGB in convolution layer: conv1
Load weights from convolution layer: conv1 (3, 3, 3, 32)
Assign variable: MobileNets/block2/conv_dw/depthwise_weights (3, 3, 32, 1)
Load weights from convolution layer: conv2_1/dw (3, 3, 1, 32)
Assign variable: MobileNets/block2/conv_pw/weights (1, 1, 32, 64)
Load weights from convolution layer: conv2_1/sep (1, 1, 32, 64)
Assign variable: MobileNets/block3/conv_dw/depthwise_weights (3, 3, 64, 1)
Load weights from convolution layer: conv2_2/dw (3, 3, 1, 64)
Assign variable: MobileNets/block3/conv_pw/weights (1, 1, 64, 128)
Load weights from convolution layer: conv2_2/sep (1, 1, 64, 128)
Assign variable: MobileNets/block4/conv_dw/depthwise_weights (3, 3, 128, 1)
Load weights from convolution layer: conv3_1/dw (3, 3, 1, 128)
Assign variable: MobileNets/block4/conv_pw/weights (1, 1, 128, 128)
Load weights from convolution layer: conv3_1/sep (1, 1, 128, 128)
Assign variable: MobileNets/blo

In [14]:
def get_bn_params(idx, caffemodel_params, dtype=tf.float32):
    # Find the correct layer.
    layers = caffemodel_params.layer
    i = -1
    j = 0    
    while j < len(layers):
        if layers[j].type == 'BatchNorm':
            i += 1
        if i == idx:
            break
        j += 1
    l_bn = layers[j]
    l_scale =  layers[j+1]
    # BN moving averages.
    mean, variance, scale = (np.array(l_bn.blobs[0].data), 
                             np.array(l_bn.blobs[1].data), 
                             np.array(l_bn.blobs[2].data))
    mean = mean / scale
    variance = variance / scale
    # Scaling parameters.
    gamma, beta = (np.array(l_scale.blobs[0].data), 
                   np.array(l_scale.blobs[1].data))
    print('Load Batch Norm parameters:', l_bn.name, mean.shape)
    return mean, variance, beta, gamma

i = 0
idx = 0
var_names_to_values = {}
variables = tf.global_variables()
while i < len(variables):
    v = variables[i]
    name = v.op.name
    if 'BatchNorm' in name and 'beta' in name:
        print('Assign variable:', v.op.name, v.get_shape())
        mean, variance, beta, gamma = get_bn_params(idx, caffemodel_params)
        var_names_to_values[variables[i].op.name] = beta
        i += 1
        var_names_to_values[variables[i].op.name] = gamma
        i += 1
        var_names_to_values[variables[i].op.name] = mean
        i += 1
        var_names_to_values[variables[i].op.name] = variance
        idx += 1
    i += 1

# Assign BN parameters...
assign_op, feed_dict = tf.contrib.framework.assign_from_values(var_names_to_values)
isess.run(assign_op, feed_dict=feed_dict)

Assign variable: MobileNets/conv1/BatchNorm/beta (32,)
Load Batch Norm parameters: conv1/bn (32,)
Assign variable: MobileNets/block2/conv_dw/BatchNorm/beta (32,)
Load Batch Norm parameters: conv2_1/dw/bn (32,)
Assign variable: MobileNets/block2/conv_pw/BatchNorm/beta (64,)
Load Batch Norm parameters: conv2_1/sep/bn (64,)
Assign variable: MobileNets/block3/conv_dw/BatchNorm/beta (64,)
Load Batch Norm parameters: conv2_2/dw/bn (64,)
Assign variable: MobileNets/block3/conv_pw/BatchNorm/beta (128,)
Load Batch Norm parameters: conv2_2/sep/bn (128,)
Assign variable: MobileNets/block4/conv_dw/BatchNorm/beta (128,)
Load Batch Norm parameters: conv3_1/dw/bn (128,)
Assign variable: MobileNets/block4/conv_pw/BatchNorm/beta (128,)
Load Batch Norm parameters: conv3_1/sep/bn (128,)
Assign variable: MobileNets/block5/conv_dw/BatchNorm/beta (128,)
Load Batch Norm parameters: conv3_2/dw/bn (128,)
Assign variable: MobileNets/block5/conv_pw/BatchNorm/beta (256,)
Load Batch Norm parameters: conv3_2/sep/bn

In [21]:
# Last fully connected layer.
i_caffe = -5
i_tf = -2
# FC weights and bias.
layer = caffemodel_params.layer[i_caffe]
# w = np.array(layer.blobs[0].data)
# w = np.reshape(w, layer.blobs[0].shape.dim)
# w = np.squeeze(np.transpose(w))
b = np.array(layer.blobs[1].data)
b = np.reshape(b, layer.blobs[1].shape.dim)

# Assign variables.
variables = tf.global_variables()
var_names_to_values = {
#     variables[i_tf].op.name: w,
    variables[i_tf+1].op.name: b,
}
# Assign...
assign_op, feed_dict = tf.contrib.framework.assign_from_values(var_names_to_values)
isess.run(assign_op, feed_dict=feed_dict)

In [22]:
# # tf.trainable_variables()
# tf.global_variables()

In [23]:
# Save model in checkpoint.
saver = tf.train.Saver()
ckpt_path = caffe_filename.replace('.caffemodel', '.ckpt')
saver.save(isess, ckpt_path, write_meta_graph=False)

'../checkpoints/mobilenets.ckpt'

In [25]:
# isess.run(tf.global_variables()[-1])

In [18]:
tf.__version__

'1.1.0'

In [19]:
tf.float32.min

-3.4028235e+38

In [20]:
tf.global_variables()

[<tf.Variable 'global_step:0' shape=() dtype=int64_ref>,
 <tf.Variable 'MobileNets/conv1/weights:0' shape=(3, 3, 3, 32) dtype=float32_ref>,
 <tf.Variable 'MobileNets/conv1/BatchNorm/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'MobileNets/conv1/BatchNorm/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'MobileNets/conv1/BatchNorm/moving_mean:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'MobileNets/conv1/BatchNorm/moving_variance:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'MobileNets/block2/conv_dw/depthwise_weights:0' shape=(3, 3, 32, 1) dtype=float32_ref>,
 <tf.Variable 'MobileNets/block2/conv_dw/BatchNorm/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'MobileNets/block2/conv_dw/BatchNorm/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'MobileNets/block2/conv_dw/BatchNorm/moving_mean:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'MobileNets/block2/conv_dw/BatchNorm/moving_variance:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'MobileNets/bl